In [11]:
import xml.dom.minidom
import os
from shutil import copy2
import json

# coco or genome or karpathy's coco
dataset = 'karp'

with open('imgname_train2014.json', 'wb') as train_img_name, \
    open('imgname_val2014.json', 'wb') as val_img_name, \
    open('captions_train2014.json', 'wb') as train_captions, \
    open('captions_val2014.json', 'wb') as val_captions, \
    open('imgs_train_path.txt', 'wb') as train_path, \
    open('imgs_val_path.txt', 'wb') as val_path:        
        
    if dataset in ('coco', 'genome'):
        train_img_name.write('{\n \"images\": [\n')
        train_captions.write('\"annotations\": [\n')
        val_img_name.write('{\n \"images\": [\n')
        val_captions.write('\"annotations\": [\n')
        # we also need the following
        #res.dataset['info']
        #res.dataset['type']
        #res.dataset['licenses']
    else:
        # init dict data
        json_data = {}
        json_data['images'] = []
        json_data['dataset'] = 'coco'
        
    
    i, j = 0, 0
    
    for xmlfile in os.listdir('../lung-report/ecgen-radiology/'): 
        print xmlfile

        DomTree = xml.dom.minidom.parse('../lung-report/ecgen-radiology/'+xmlfile)
        eCitation = DomTree.childNodes[0]
        parentImages = eCitation.getElementsByTagName('parentImage')

        MedlineCitation = eCitation.getElementsByTagName('MedlineCitation')[0]
        Abstract = MedlineCitation.getElementsByTagName('Abstract')[0]

        
        try:
            # COMPARISON = Abstract.getElementsByTagName('AbstractText')[0].firstChild.data
            # INDICATION = Abstract.getElementsByTagName('AbstractText')[1].firstChild.data
            FINDINGS = Abstract.getElementsByTagName('AbstractText')[2].firstChild.data
            # IMPRESSION = Abstract.getElementsByTagName('AbstractText')[3].firstChild.data
            
            # TRUNCATE LONG DESCRIPTION
            if len(FINDINGS.split(' ')) > 60:
                FINDINGS = ' '.join(FINDINGS.split(' ')[:60])
            
            j += 1 # report id

            for parentImage in parentImages:
                
                i += 1 # image id
                
                # special dict for karp
                if dataset == 'karp':
                    sent = {}
                    sent['imgid'], sent['sentids'] = i, i
                    sent['raw'] = FINDINGS
                    for char in  ['!', '?', '\'', ',', ';', '.', ':']:
                        FINDINGS = FINDINGS.replace(char, '')
                    sent['tokens'] = [x.lower() for x in FINDINGS.split(' ')]
                    
                    img = {}
                    img['filename'] = parentImage.attributes.items()[0][1] + '.png'
                    img['imgid'], img['sentids'], img['cocoid'] = i, [i], i
                    img['sentences'] = [sent]
                    
                
                
                # 70% 4532, 80% 5178, 90% 5825
                if i < 5825:               
                    
                    if dataset == 'coco':
                        train_img_name.write('{' + '\"file_name\": \"' + parentImage.attributes.items()[0][1] + '.png\"' +\
                                             ',' + '\"id\": ' + str(i) + '},')
                        train_captions.write('{' + '\"image_id\": ' + str(i) + ',' +
                                             '\"id\": ' + str(j) + ',' + '\"caption\": ' + '\"' + FINDINGS + '\"' + '},')
                        
                        #copy2('../lung-report/image/'+ parentImage.attributes.items()[0][1] + '.png', './image/train2014/')
                        
                    elif dataset == 'genome':
                        train_path.write('../lung-report/image/' + parentImage.attributes.items()[0][1] + '.png' + '\n')                        
                        train_captions.write('{' + '\"image_id\": ' + parentImage.attributes.items()[0][1] + ',' +
                                             '\"id\": ' + str(j) + ',' + '\"paragraph\": ' + '\"' + FINDINGS + '\"' + '},')
                        
                    elif dataset == 'karp': 
                        # sentence level, bottom up filling
                        img['split'] = 'train'
                        img['filepath'] = "train2014"
                        json_data['images'].append(img)
                        
                        copy2('../lung-report/image/'+ parentImage.attributes.items()[0][1] + '.png', './image/train2014/')
                    
                else:
                    
                    if dataset == 'coco':
                        val_img_name.write('{' + '\"file_name\": \"' + parentImage.attributes.items()[0][1] + '.png\"' +\
                                           ',' + '\"id\": ' + str(i) + '},')
                        val_captions.write('{' + '\"image_id\": ' + str(i) + ',' +
                                             '\"id\": ' + str(j) + ',' + '\"caption\": ' + '\"' + FINDINGS + '\"' + '},')
                        
                        #copy2('../lung-report/image/'+ parentImage.attributes.items()[0][1] + '.png', './image/val2014/')
                        
                    elif dataset == 'genome':
                        val_path.write('../lung-report/image/' + parentImage.attributes.items()[0][1] + '.png' + '\n')
                        val_captions.write('{' + '\"image_id\": ' + parentImage.attributes.items()[0][1] + ',' +
                                             '\"id\": ' + str(j) + ',' + '\"paragraph\": ' + '\"' + FINDINGS + '\"' + '},')

                    elif dataset == 'karp':                        
                        img['split'] = 'test' # also use for val
                        img['filepath'] = "test2014"
                        json_data['images'].append(img)
                        
                        # duplicate for val
                        img['split'] = 'val'
                        img['filepath'] = "val2014"
                        json_data['images'].append(img)
                        
                        copy2('../lung-report/image/'+ parentImage.attributes.items()[0][1] + '.png', './image/val2014/')
                    
                print parentImage.attributes.items()[0][1] + '.png'
        
        except AttributeError:
            print 'This xml has no findings'
            pass

        
    if dataset in ('coco', 'genome'):
        train_img_name.write(']')
        val_img_name.write(']')
        train_captions.write(']}')
        val_captions.write(']}')   
    else:
        json.dump(json_data, train_captions)
        

In [10]:
%mv captions_train2014.json ./data/dataset_coco.json 
%rm imgs*.txt
%rm imgname*.json
%cp -r ./image/val2014 ./image/test2014
